In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import os,gc
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc,roc_auc_score 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
torch.cuda.set_device(5)
print (torch.cuda.current_device())

5


In [2]:
#preparing the trainset and  testset
img_path = '/data/fjsdata/NIH-CXR/images/images/' 
trN, trI, trY = [],[],[]
with open('/data/fjsdata/NIH-CXR/chexnet_dataset/train.txt', "r") as file_descriptor: #tarinset
    lines = file_descriptor.readlines()
    for line in lines:
        try:
            line_items = line.split()
            image_name = line_items[0].split('/')[1]
            trN.append(image_name)
            image_label = line_items[1:]  # 14 labels from index 2
            image_label = [int(i) for i in image_label]  
            trY.append(np.array(image_label))
            img = cv2.resize(cv2.imread(os.path.join(img_path, image_name)).astype(np.float32), (256, 256))#(256,256,3)
            #img = img[16:224, 16:224] #center crop 224
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX) #normalization
            trI.append(img)
        except:
            print(image_name+":"+str(os.path.join(img_path, image_name)))
        sys.stdout.write('\r{} / {} '.format(len(trN),78468))
        sys.stdout.flush()
trI = np.array(trI)
trY = np.array(trY)   
print('The length of trainset is %d'%len(trN))
        
valN, valI, valY = [],[],[]
with open('/data/fjsdata/NIH-CXR/chexnet_dataset/val.txt', "r") as file_descriptor: #valset
    lines = file_descriptor.readlines()
    for line in lines:
        try:
            line_items = line.split()
            image_name = line_items[0].split('/')[1]
            valN.append(image_name)
            image_label = line_items[1:]  # 14 labels from index 2
            image_label = [int(i) for i in image_label]  
            valY.append(np.array(image_label))
            img = cv2.resize(cv2.imread(os.path.join(img_path, image_name)).astype(np.float32), (256, 256))#(256,256,3)
            #img = img[16:224, 16:224] #center crop 224
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX) #normalization
            valI.append(img)
        except:
            print(image_name+":"+str(os.path.join(img_path, image_name)))
        sys.stdout.write('\r{} / {} '.format(len(valN),11219))
        sys.stdout.flush()
valI = np.array(valI)
valY = np.array(valY) 
print('The length of validset is %d'%len(valN))

teN, teI, teY = [],[],[]
with open('/data/fjsdata/NIH-CXR/chexnet_dataset/test.txt', "r") as file_descriptor: #testset
    lines = file_descriptor.readlines()
    for line in lines:
        try:
            line_items = line.split()
            image_name = line_items[0].split('/')[1]
            teN.append(image_name)
            image_label = line_items[1:]  # 14 labels from index 2
            image_label = [int(i) for i in image_label]  
            teY.append(np.array(image_label))
            img = cv2.resize(cv2.imread(os.path.join(img_path, image_name)).astype(np.float32), (256, 256))#(256,256,3)
            #img = img[16:224, 16:224] #center crop 224
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX) #normalization
            teI.append(img)
        except:
            print(image_name+":"+str(os.path.join(img_path, image_name)))
        sys.stdout.write('\r{} / {} '.format(len(teN),22433))
        sys.stdout.flush()
teI = np.array(teI)
teY = np.array(teY)    
print('The length of testset is %d'%len(teN))

78468 / 78468 The length of trainset is 78468
11219 / 11219 The length of validset is 11219
22433 / 22433 The length of testset is 22433


In [4]:
# construct model
class DenseNet121(nn.Module):
    def __init__(self, num_classes, is_pre_trained):
        super(DenseNet121, self).__init__()
        self.dense_net_121 = torchvision.models.densenet121(pretrained=is_pre_trained)
        num_fc_kernels = self.dense_net_121.classifier.in_features
        self.dense_net_121.classifier = nn.Sequential(nn.Linear(num_fc_kernels, num_classes), nn.Sigmoid())

    def forward(self, x):
        x = self.dense_net_121(x)
        return x


class DenseNet169(nn.Module):
    def __init__(self, num_classes, is_pre_trained):
        super(DenseNet169, self).__init__()
        self.dense_net_169 = torchvision.models.densenet169(pretrained=is_pre_trained)
        num_fc_kernels = self.dense_net_169.classifier.in_features
        self.dense_net_169.classifier = nn.Sequential(nn.Linear(num_fc_kernels, num_classes), nn.Sigmoid())

    def forward(self, x):
        x = self.dense_net_169(x)
        return x


class DenseNet201(nn.Module):
    def __init__(self, num_classes, is_pre_trained):
        super(DenseNet201, self).__init__()
        self.dense_net_201 = torchvision.models.densenet201(pretrained=is_pre_trained)
        num_fc_kernels = self.dense_net_201.classifier.in_features
        self.dense_net_201.classifier = nn.Sequential(nn.Linear(num_fc_kernels, num_classes), nn.Sigmoid())

    def forward(self, x):
        x = self.dense_net_201(x)
        return x

In [17]:
#model training
N_CLASSES = 14 #class numbers
model = DenseNet121(num_classes=N_CLASSES, is_pre_trained=True).cuda()#initialize model
#model = torch.nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5, 6, 7]).cuda()# make model available multi GPU cores training
torch.backends.cudnn.benchmark = True  # improve train speed slightly
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5, mode='min')
criterion = torch.nn.BCELoss()
#train model
best_net, best_loss = None, float('inf')
batchSize = 20 #'Batch Size': 32
for epoch in range(10):#'Max Epoch': 1000
    num_batches = len(trY) // batchSize + 1
    model.train()  # set network as train mode
    with torch.autograd.enable_grad():
        for i in range(num_batches):
            optimizer.zero_grad()#grad vanish
            min_idx = i * batchSize
            max_idx = np.min([len(trY), (i+1)*batchSize])
            I_batch = torch.from_numpy(trI[min_idx:max_idx]).type(torch.FloatTensor).cuda()
            y_batch = torch.from_numpy(trY[min_idx:max_idx]).type(torch.FloatTensor).cuda()
            #forword
            y_outputs = model(I_batch.permute(0, 3, 1, 2))#permute the dims of matrix
            #loss
            loss = criterion(y_outputs, y_batch)
            loss.backward()
            #update parameters
            optimizer.step()
            sys.stdout.write('\r {} / {} : train loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
            sys.stdout.flush()     
    #validation process
    loss_val = []
    mean_loss_tensor = 0.
    num_batches = len(valY) // batchSize  +1
    model.eval()  # set network as eval mode without BN & Dropout
    with torch.autograd.no_grad():
        for j in range(num_batches):
            min_idx = j * batchSize
            max_idx = np.min([len(valY), (j+1)*batchSize])
            I_batch = torch.from_numpy(valI[min_idx:max_idx]).type(torch.FloatTensor).cuda()
            y_batch = torch.from_numpy(valY[min_idx:max_idx]).type(torch.FloatTensor).cuda()
            y_outputs = model(I_batch.permute(0, 3, 1, 2))#forword
            curr_loss = criterion(y_outputs, y_batch)
            sys.stdout.write('\r {} / {} : validation loss = {}'.format(j + 1, num_batches, float('%0.6f'%curr_loss.item()) ) )
            sys.stdout.flush()  
            mean_loss_tensor += curr_loss  # tensor op.
            loss_val.append(curr_loss.item())
    mean_loss_tensor = mean_loss_tensor / len(valY)  # tensor
    scheduler.step(mean_loss_tensor.item())
    print("Eopch: %5d val_loss = %.6f" % (epoch + 1, np.mean(loss_val)))
    if np.mean(loss_val) < best_loss:
        best_loss = np.mean(loss_val)
        best_net = copy.deepcopy(model)        
print("best_loss = %.6f" % (best_loss))
model = model.cpu()#release gpu memory
I_batch = I_batch.cpu()
y_batch = y_batch.cpu()
torch.cuda.empty_cache()

 561 / 561 : validation loss = 0.156511Eopch:     1 val_loss = 0.180127
 561 / 561 : validation loss = 0.156758Eopch:     2 val_loss = 0.176577
 561 / 561 : validation loss = 0.156105Eopch:     3 val_loss = 0.175030
 561 / 561 : validation loss = 0.147324Eopch:     4 val_loss = 0.171894
 561 / 561 : validation loss = 0.149296Eopch:     5 val_loss = 0.170087
 561 / 561 : validation loss = 0.150558Eopch:     6 val_loss = 0.166671
 561 / 561 : validation loss = 0.154989Eopch:     7 val_loss = 0.168227
 561 / 561 : validation loss = 0.145486Eopch:     8 val_loss = 0.166358
 561 / 561 : validation loss = 0.143663Eopch:     9 val_loss = 0.165255
 561 / 561 : validation loss = 0.140894Eopch:    10 val_loss = 0.164218
best_loss = 0.164218


In [18]:
#performance of testset
# initialize the ground truth and output tensor
gt = torch.FloatTensor().cuda()
pred = torch.FloatTensor().cuda()
num_batches = len(teY) // batchSize  +1
best_net.eval()  # set network as eval mode without BN & Dropout
with torch.autograd.no_grad():
    for i in range(num_batches):
        min_idx = i * batchSize
        max_idx = np.min([len(teY), (i+1)*batchSize])
        I_batch = torch.from_numpy(teI[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        y_batch = torch.from_numpy(teY[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        gt = torch.cat((gt, y_batch), 0)
        y_outputs = best_net(I_batch.permute(0, 3, 1, 2))#forword
        pred = torch.cat((pred, y_outputs.data), 0)
        sys.stdout.write('\r {} / {} '.format(i, num_batches))
        sys.stdout.flush()
    
CLASS_NAMES = ['Atelectasis', 'Cardiomegaly', 'Effusion','Infiltration', 'Mass', 'Nodule', 'Pneumonia', \
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'] 
def compute_AUCs(gt, pred):
    AUROCs = []
    gt_np = gt.cpu().numpy()
    pred_np = pred.cpu().numpy()
    for i in range(N_CLASSES):
        AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
    return AUROCs

AUROCs = compute_AUCs(gt, pred)
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.4f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
    print('The AUROC of {} is {:.4f}'.format(CLASS_NAMES[i], AUROCs[i]))

 1121 / 1122 The average AUROC is 0.7329
The AUROC of Atelectasis is 0.7368
The AUROC of Cardiomegaly is 0.7850
The AUROC of Effusion is 0.8363
The AUROC of Infiltration is 0.6619
The AUROC of Mass is 0.6859
The AUROC of Nodule is 0.6468
The AUROC of Pneumonia is 0.6766
The AUROC of Pneumothorax is 0.7488
The AUROC of Consolidation is 0.7708
The AUROC of Edema is 0.8367
The AUROC of Emphysema is 0.7343
The AUROC of Fibrosis is 0.7250
The AUROC of Pleural_Thickening is 0.6893
The AUROC of Hernia is 0.7264


In [29]:
#https://github.com/marvis/pytorch-yolo2/blob/master/FocalLoss.py
#https://github.com/clcarwin/focal_loss_pytorch/blob/master/focalloss.py
class FocalLoss(nn.Module):
    #Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, out, y):
        #y = y.view(-1,1)
        y_index = torch.nonzero(y)  ???
        logpt = F.log_softmax(out,dim=1)#default ,dim=1
        logpt = logpt.gather(1,y_index)# dim=1, index=y, max
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=out.data.type():
                self.alpha = self.alpha.type_as(out.data)
            at = self.alpha.gather(0,y.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [10]:
# normalize data with ImageNet mean and standard deviation
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform_list = list()# compose transform operations
transform_list.append(transforms.Resize(256))
transform_list.append(torchvision.transforms.RandomHorizontalFlip())
transform_list.append(transforms.TenCrop(244))
transform_list.append(normalize)
transform_list.append(transforms.ToTensor())
transform_sequence = transforms.Compose(transform_list)
img = cv2.resize(cv2.imread('/data/fjsdata/NIH-CXR/images/images/00001522_000.png').astype(np.float32), (256, 256))
#unique, counts = np.unique(img, return_counts=True)
#print(np.asarray((unique, counts)).T)
img = img[16:224, 16:224]
img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX)
#unique, counts = np.unique(img, return_counts=True)
#print(np.asarray((unique, counts)).T)
print(img.shape)

(208, 208, 3)


In [ ]:
#preparing the trainset and  testset
img_path = '/data/fjsdata/NIH-CXR/images/images/' 
trN, trI, trY = [],[],[]
with open('/data/fjsdata/NIH-CXR/chexnet_dataset/train.txt', "r") as file_descriptor: #tarinset
    lines = file_descriptor.readlines()
    for line in lines:
        try:
            line_items = line.split()
            image_name = line_items[0].split('/')[1]
            trN.append(image_name)
            image_label = line_items[1:]  # 14 labels from index 2
            image_label = [int(i) for i in image_label]  
            if np.array(image_label).sum()==0: #normal
                trY.append(0)
            else: trY.append(1)#abnormality
            img = cv2.resize(cv2.imread(os.path.join(img_path, image_name)).astype(np.float32), (256, 256))#(256,256,3)
            #img = img[16:224, 16:224] #center crop 224
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX) #normalization
            trI.append(img)
        except:
            print(image_name+":"+str(os.path.join(img_path, image_name)))
        sys.stdout.write('\r{} / {} '.format(len(trN),78468))
        sys.stdout.flush()
trI = np.array(trI)
trY = np.array(trY)   
print('The length of trainset is %d'%len(trN))
        
valN, valI, valY = [],[],[]
with open('/data/fjsdata/NIH-CXR/chexnet_dataset/val.txt', "r") as file_descriptor: #valset
    lines = file_descriptor.readlines()
    for line in lines:
        try:
            line_items = line.split()
            image_name = line_items[0].split('/')[1]
            valN.append(image_name)
            image_label = line_items[1:]  # 14 labels from index 2
            image_label = [int(i) for i in image_label]  
            if np.array(image_label).sum()==0: #normal
                valY.append(0)
            else: valY.append(1) #abnormality
            img = cv2.resize(cv2.imread(os.path.join(img_path, image_name)).astype(np.float32), (256, 256))#(256,256,3)
            #img = img[16:224, 16:224] #center crop 224
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX) #normalization
            valI.append(img)
        except:
            print(image_name+":"+str(os.path.join(img_path, image_name)))
        sys.stdout.write('\r{} / {} '.format(len(valN),11219))
        sys.stdout.flush()
valI = np.array(valI)
valY = np.array(valY) 
print('The length of validset is %d'%len(valN))

teN, teI, teY = [],[],[]
with open('/data/fjsdata/NIH-CXR/chexnet_dataset/test.txt', "r") as file_descriptor: #testset
    lines = file_descriptor.readlines()
    for line in lines:
        try:
            line_items = line.split()
            image_name = line_items[0].split('/')[1]
            teN.append(image_name)
            image_label = line_items[1:]  # 14 labels from index 2
            image_label = [int(i) for i in image_label]  
            if np.array(image_label).sum()==0: #normal
                teY.append(0)
            else: teY.append(1) #abnormality
            img = cv2.resize(cv2.imread(os.path.join(img_path, image_name)).astype(np.float32), (256, 256))#(256,256,3)
            #img = img[16:224, 16:224] #center crop 224
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX) #normalization
            teI.append(img)
        except:
            print(image_name+":"+str(os.path.join(img_path, image_name)))
        sys.stdout.write('\r{} / {} '.format(len(teN),22433))
        sys.stdout.flush()
teI = np.array(teI)
teY = np.array(teY)    
print('The length of testset is %d'%len(teN))